### GENERATE A RANDOM BPMN

In [ ]:
#################
# IMPORTS
#################
import requests
import re
import sys
sys.path.append('src')
from paco.parser.parse_tree import ParseTree
from paco.execution_tree.execution_tree import ExecutionTree
from paco.explainer.bdd.bdds import bdds_from_json

#################
# HEADERS
#################
headers = {
	"Content-Type": "application/json",
}
url = "http://127.0.0.1:8000/"

In [ ]:
###########################
# Define the BPMN STRUCTURE
###########################
expression = 'T1, <[L1] T2 >'

impacts_range = [1, 50]
duration_range = [1, 100]
delay_range = [0, 10]

tasks = sorted(set(re.findall(r'T\d+', expression)))
natures = sorted(set(re.findall(r'N\d+', expression)))
choices = sorted(set(re.findall(r'C\d+', expression)))
bpmn = {
	'expression': expression,
	'impacts': {'T1': [11, 15], 'T2': [4, 2]},
	'durations': {'T1': [0, 100], 'T2': [0, 100]},
	'impacts_names': ['cost', 'hours'],
	'probabilities': {},
	'names': {'L1': 'L1'},
	'delays': {},
	'loop_probability': {'L1': 0.5},
	'loop_round': {'L1': 10}, "h": 0,
}

### Check if the expression compliy with the defined grammmar

In [ ]:
resp = requests.get(f'{url}check_correct_process_expression', params={'expression': bpmn['expression']},  headers=headers)
if resp.status_code != 200:
	print('Error in the request', resp.text)
elif resp.text == 'true':
	print('BPMN grammar is correct')
else:
	print('BPMN grammar is incorrect')

### Print Diagram

In [ ]:
import graphviz
from IPython.display import display, SVG

response = requests.post(f'{url}create_sese_diagram', json={"bpmn": bpmn},  headers=headers)

if response.status_code == 200:
	display(SVG(graphviz.Source(response.json()['graph']).pipe(format="svg")))
else:
	print(f"Error: {response.status_code}")
	print(f"Response: {response.text}")

### Sample Expected Impact

In [ ]:
response = requests.get(f'{url}get_parse_tree', json={"bpmn": bpmn},  headers=headers)

if response.status_code == 200:
	parse_tree, pending_choices, pending_natures = ParseTree.from_json(
		response.json()['parse_tree'],
		impact_size=len(bpmn['impacts_names']),
		non_cumulative_impact_size=0)

	bound = parse_tree.sample_expected_impact()
	print(f"Sampled Expected Impact: {bound}")

	display(SVG(graphviz.Source(parse_tree.to_dot()).pipe(format="svg")))

else:
	print(f"Error: {response.status_code}")
	print(f"Response: {response.text}")

### Create the Execution tree

In [ ]:
response = requests.get(f'{url}calc_strategy_general', json={'bpmn': bpmn, 'bound': bound, 'algo': 'paco'}, headers=headers)

if response.status_code != 200:
	print(f"Error: {response.status_code}, {response.text}")
else:
	response = response.json()
	print(response["result"])
	parse_tree, pending_choice, pending_natures = ParseTree.from_json(response["parse_tree"], len(bpmn["impacts_names"]), 0)
	dot = parse_tree.to_dot()
	display(SVG(graphviz.Source(dot).pipe(format="svg")))

	execution_tree = ExecutionTree.from_json(parse_tree, response["execution_tree"], bpmn["impacts_names"])

	frontier_solution_id = set()
	if "frontier_solution" in response: # Solution Found
		frontier_solution_id = set(map(int, response["frontier_solution"].strip("[]").split(",")))

	# With frontier node in blue
	dot = execution_tree.to_dot(state=True, executed_time=False, diff=True, frontier=frontier_solution_id)
	display(SVG(graphviz.Source(dot).pipe(format="svg")))


	if "strategy_tree" in response:
		explained_choices = bdds_from_json(parse_tree, response["bdds"])
		print("1 is dashed line of BPMN or Parse Tree")
		for choice, bdd in explained_choices.items():
			print(f"{choice.name} : {bdd.typeStrategy}")
			svg_data = graphviz.Source(bdd.bdd_to_dot()).pipe(format="svg")
			display(SVG(svg_data))


		strategy_tree = ExecutionTree.from_json(parse_tree, response["strategy_tree"], bpmn["impacts_names"], explained_choices)

		#TREE_STATE
		dot = strategy_tree.to_dot(state=True, executed_time=False, diff=True)
		display(SVG(graphviz.Source(dot).pipe(format="svg")))